In [43]:
new_tweet = "they went to war when russia invaded ukraine they believed in freedom the right to live in their godgiven land they laid down their lives not only for ukraine but for all of europe rip mykhailo lompas 31 mykola los liubomyr lutsak roman lutsyk 28 "
hashtags = ["ArmUkraineNow"]

In [44]:
import re

def tweet_processing(tweet,hashtags):
    import contractions
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem.wordnet import WordNetLemmatizer 
    from gensim.parsing.preprocessing import STOPWORDS
    lemmatizer = WordNetLemmatizer()

    words = new_tweet.split(" ")
    sentence = []
    for word in words:
        try:
            word = contractions.fix(word)
        except:
            pass
        finally:
            sentence.append(word)
    processed = " ".join(sentence)

    processed = " ".join([w.lower() for w in processed.split(" ")])

    processed = re.sub(r"@\S+","",processed)
    processed = re.sub(r"#\S+","",processed)
    processed = re.sub(" amp "," ",processed)
    processed = re.sub(r"http\S+","",processed)
    processed = re.sub(r'[^\w\s]', '', processed)
    processed = re.sub(r'[^a-zA-Z\s]', '', processed)
    processed = re.sub(r'\d+', '', processed)

    processed = word_tokenize(processed)

    processed = [lemmatizer.lemmatize(token) for token in processed if token not in STOPWORDS and len(token) > 1]

    processed = [w for w in processed if re.search('^[a-z]+$',w)]
    
    import pickle as pkl

    with open("final_lda_model.pkl", "rb") as fp:
        lda_model = pkl.load(fp)
    with open("dictionary.pkl", "rb") as fp:
        unigram_dictionary = pkl.load(fp)
    with open("relevance_model.pkl", "rb") as fp:
        relevance_model = pkl.load(fp)
    with open("sentiment_model.pkl", "rb") as fp:
        sentiment_model = pkl.load(fp)
        
    import gensim
    from gensim import corpora

    gensim_corpus = unigram_dictionary.doc2bow(processed)
    topic_score = lda_model.get_document_topics(gensim_corpus)
    max = 0
    best = None
    for i in range(0,len(topic_score)):
        if topic_score[i][1] > max:
            max = topic_score[i][1]
            best = topic_score[i][0]

    if best == 0:
        best = "Military Actions and Operations"
    elif best == 1:
        best = "Civilian Support and Information Sharing"
    elif best == 2:
        best = "Broad Overview of the Conflict"
    elif best == 3:
        best = "Strategic Military Aspects"
    else:
        best = "International Support and Public Opinion"
        
    vector = unigram_dictionary.doc2bow(processed)
    sent_as_dict = {id:1 for (id, tf) in vector}
    for tag in hashtags:
        sent_as_dict[tag] = 1
        
    relevance = relevance_model.classify(sent_as_dict)
    
    if relevance == "Relevant":
        vector = unigram_dictionary.doc2bow(processed)
        sent_as_dict = {id:1 for (id, tf) in vector}
        for tag in hashtags:
            sent_as_dict[tag] = 1
        sent_as_dict[best] = 1
        sentiment = sentiment_model.classify(sent_as_dict)
    else: 
        sentiment = "Irrelevant"
        
    from transformers import pipeline
    import tensorflow

    # download pre-trained emotion classification model
    emorobertamodel = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")
    
    emotions = emorobertamodel(new_tweet)
    
    return (best,relevance,sentiment,emotions[0]["label"])

In [45]:
(best,relevance,sentiment,emotions) = tweet_processing(new_tweet,hashtags)
print(best)
print(relevance)
print(sentiment)
print(emotions)

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
module 'tensorflow' has no attribute 'data'